In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 5 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (5,363 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 198 kB/s 
     |████████████████████████████████| 465 kB 40.4 MB/s 


In [ ]:
import pandas as pd 
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_string_dtype

In [ ]:
import nltk
from konlpy.tag import Okt
from wordcloud import WordCloud
from collections import defaultdict
from collections import Counter
from nltk.corpus import stopwords

In [ ]:
plt.rc('font', family='NanumBarunGothic') 

In [ ]:
raw_data_19 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/추가데이터/NH농협캐피탈 데이터 작업_승진적합_1114.xlsx', sheet_name=0)
major = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/dataset/전공 대분류.xlsx')

In [ ]:
raw_data_19['전공'] = major['대분류']

In [ ]:
raw_data_20 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/추가데이터/NH농협캐피탈 데이터 작업_승진적합_1114.xlsx', sheet_name=1)
raw_data_21 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/추가데이터/NH농협캐피탈 데이터 작업_승진적합_1114.xlsx', sheet_name=2)

In [ ]:
raw_data_20['전공'] = major['대분류']
raw_data_21['전공'] = major['대분류']

In [ ]:
sen1_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/dataset/sentiment_19년상.xlsx')
sen2_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/dataset/sentiment_19년하.xlsx')
sen3_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/dataset/sentiment_20년상.xlsx')
sen4_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/dataset/sentiment_20년하.xlsx')
sen5_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/dataset/sentiment_21년하.xlsx')

In [ ]:
sen1 = sen1_data['사번'].unique()
sen2 = sen2_data['사번'].unique()
sen_2019 = list(set(np.append(sen1, sen2)))

In [ ]:
sen3 = sen3_data['사번'].unique()
sen4 = sen4_data['사번'].unique()
sen_2020 = list(set(np.append(sen3, sen4)))

In [ ]:
sen_2021 = sen5_data['사번'].unique()

In [ ]:
sen1_data = sen1_data.replace('positive', 1)
sen1_data = sen1_data.replace('negative', -1)
sen1_data = sen1_data.replace('neutral', 0)

In [ ]:
sen2_data = sen2_data.replace('positive', 1)
sen2_data = sen2_data.replace('negative', -1)
sen2_data = sen2_data.replace('neutral', 0)

In [ ]:
sen3_data = sen3_data.replace('positive', 1)
sen3_data = sen3_data.replace('negative', -1)
sen3_data = sen3_data.replace('neutral', 0)

In [ ]:
sen4_data = sen4_data.replace('positive', 1)
sen4_data = sen4_data.replace('negative', -1)
sen4_data = sen4_data.replace('neutral', 0)

In [ ]:
sen5_data = sen5_data.replace('positive', 1)
sen5_data = sen5_data.replace('negative', -1)
sen5_data = sen5_data.replace('neutral', 0)

In [ ]:
con_list = []
for name in sen_2019 : 
  temp_list = []
  temp_list.append(name)
  first = sen1_data.loc[sen1_data['사번']==name].reset_index(drop=True)
  half = sen2_data.loc[sen2_data['사번']==name].reset_index(drop=True)
  if (len(first)==0 and len(half)!=0) : 
    temp_list.append(half['직책수행능력'][0])
    temp_list.append(half['조직내 필요성'][0])
    temp_list.append(half['동반자 의식'][0])
  elif (len(half)==0 and len(first)!=0) : 
    temp_list.append(first['직책수행능력'][0])
    temp_list.append(first['조직내 필요성'][0])
    temp_list.append(first['동반자 의식'][0])
  else : 
    if (first['직책수행능력'][0]+half['직책수행능력'][0] > 0) : 
      temp_list.append(1)
    elif (first['직책수행능력'][0]+half['직책수행능력'][0] < 0) : 
      temp_list.append(-1)
    else : 
      temp_list.append(0)
    
    if (first['조직내 필요성'][0]+half['조직내 필요성'][0] > 0) : 
      temp_list.append(1)
    elif (first['조직내 필요성'][0]+half['조직내 필요성'][0] < 0) : 
      temp_list.append(-1)
    else : 
      temp_list.append(0)

    if (first['동반자 의식'][0]+half['동반자 의식'][0] > 0) : 
      temp_list.append(1)
    elif (first['동반자 의식'][0]+half['동반자 의식'][0] < 0) : 
      temp_list.append(-1)
    else : 
      temp_list.append(0)

  con_list.append(temp_list)

sentiment_2019 = pd.DataFrame(con_list, columns=['사번', '직책수행능력', '조직내 필요성', '동반자 의식'])

In [ ]:
con_list = []
for name in sen_2020 : 
  temp_list = []
  temp_list.append(name)
  first = sen3_data.loc[sen3_data['사번']==name].reset_index(drop=True)
  half = sen4_data.loc[sen4_data['사번']==name].reset_index(drop=True)
  if (len(first)==0 and len(half)!=0) : 
    temp_list.append(half['직책수행능력'][0])
    temp_list.append(half['조직내 필요성'][0])
    temp_list.append(half['동반자 의식'][0])
  elif (len(half)==0 and len(first)!=0) : 
    temp_list.append(first['직책수행능력'][0])
    temp_list.append(first['조직내 필요성'][0])
    temp_list.append(first['동반자 의식'][0])
  else : 
    if (first['직책수행능력'][0]+half['직책수행능력'][0] > 0) : 
      temp_list.append(1)
    elif (first['직책수행능력'][0]+half['직책수행능력'][0] < 0) : 
      temp_list.append(-1)
    else : 
      temp_list.append(0)
    
    if (first['조직내 필요성'][0]+half['조직내 필요성'][0] > 0) : 
      temp_list.append(1)
    elif (first['조직내 필요성'][0]+half['조직내 필요성'][0] < 0) : 
      temp_list.append(-1)
    else : 
      temp_list.append(0)

    if (first['동반자 의식'][0]+half['동반자 의식'][0] > 0) : 
      temp_list.append(1)
    elif (first['동반자 의식'][0]+half['동반자 의식'][0] < 0) : 
      temp_list.append(-1)
    else : 
      temp_list.append(0)

  con_list.append(temp_list)

sentiment_2020 = pd.DataFrame(con_list, columns=['사번', '직책수행능력', '조직내 필요성', '동반자 의식'])

In [ ]:
sentiment_2021 = sen5_data

In [ ]:
en_list = []

f_list = ['전문성_평균평점_상반기', '리더십_평균평점_상반기', '부하육성_평균평점_상반기',
       '관계관리_평균평점_상반기', '조직비전_평균평점_상반기', '직책수행능력_평균평점_상반기', '조직내 필요성_평균평점_상반기',
       '동반자 의식_평균평점_상반기']
h_list = ['전문성_평균평점_하반기', '리더십_평균평점_하반기', '부하육성_평균평점_하반기',
       '관계관리_평균평점_하반기', '조직비전_평균평점_하반기', '직책수행능력_평균평점_하반기', '조직내 필요성_평균평점_하반기',
       '동반자 의식_평균평점_하반기']

for i in range(len(raw_data_19)) : 
  temp = [] 
  j = 0
  for name in f_list : 
    if (type(raw_data_19[name][i])==type(str()) and type(raw_data_19[h_list[j]][i])!=type(str())) : 
      temp.append(raw_data_19[h_list[j]][i])
    elif (type(raw_data_19[name][i])!=type(str()) and type(raw_data_19[h_list[j]][i])==type(str())) : 
      temp.append(raw_data_19[name][i])
    elif (type(raw_data_19[name][i])==type(str()) and type(raw_data_19[h_list[j]][i])==type(str())) : 
      temp.append("No")
    else : 
      avg = (raw_data_19[name][i]+raw_data_19[h_list[j]][i])/2
      temp.append(avg)
    j += 1
  en_list.append(temp)

In [ ]:
raw_data_19[['전문성_평균평점', '리더십_평균평점', '부하육성_평균평점', '관계관리_평균평점', '조직비전_평균평점', '직책수행능력_평균평점', '조직내 필요성_평균평점', '동반자 의식_평균평점']] = en_list
raw_data_19 = raw_data_19.drop(f_list, axis=1)
raw_data_19 = raw_data_19.drop(h_list, axis=1)

In [ ]:
tmp = pd.DataFrame(columns=raw_data_19.columns)
for name in sen_2019 : 
  temp = raw_data_19.loc[raw_data_19['사번']==name]
  tmp = pd.concat([tmp, temp], ignore_index=True, axis=0)

In [ ]:
tmp = tmp.join(sentiment_2019.set_index('사번')['직책수행능력'], on='사번')
tmp = tmp.join(sentiment_2019.set_index('사번')['조직내 필요성'], on='사번')
tmp = tmp.join(sentiment_2019.set_index('사번')['동반자 의식'], on='사번')

In [ ]:
data_2019 = tmp
print(len(data_2019))

44


In [ ]:
temp_19 = raw_data_19
b = data_2019['사번'].tolist()

for i in range(len(raw_data_19)) : 
  if(raw_data_19['사번'][i] in b) : 
    temp_19 = temp_19.drop([i], axis=0)

temp_19 = temp_19.loc[temp_19['당년 역량평가등급']!='미입사']
temp_19 = temp_19.loc[temp_19['당년 역량평가등급']!='미평가']

In [ ]:
table = []

for i in range(len(raw_data_19)) : 
  sen = []
  if (raw_data_19['사번'][i] in data_2019['사번'].unique()) : 
    temp = data_2019.loc[data_2019['사번']==raw_data_19['사번'][i]].reset_index(drop=True)
    sen.append(temp['직책수행능력'][0])
    sen.append(temp['조직내 필요성'][0])
    sen.append(temp['동반자 의식'][0])
  else : 
    sen.append('No')
    sen.append('No')
    sen.append('No')
  table.append(sen)

In [ ]:
raw_data_19[['직책수행능력', '조직내 필요성', '동반자 의식']] = table

In [ ]:
raw_data_20['전공'] = major['대분류']

In [ ]:
en_list = []

f_list = ['전문성_평균평점_상반기', '리더십_평균평점_상반기', '부하육성_평균평점_상반기',
       '관계관리_평균평점_상반기', '조직비전_평균평점_상반기', '직책수행능력_평균평점_상반기', '조직내 필요성_평균평점_상반기',
       '동반자 의식_평균평점_상반기']
h_list = ['전문성_평균평점_하반기', '리더십_평균평점_하반기', '부하육성_평균평점_하반기',
       '관계관리_평균평점_하반기', '조직비전_평균평점_하반기', '직책수행능력_평균평점_하반기', '조직내 필요성_평균평점_하반기',
       '동반자 의식_평균평점_하반기']

for i in range(len(raw_data_20)) : 
  temp = [] 
  j = 0
  for name in f_list : 
    if (type(raw_data_20[name][i])==type(str()) and type(raw_data_20[h_list[j]][i])!=type(str())) : 
      temp.append(raw_data_20[h_list[j]][i])
    elif (type(raw_data_20[name][i])!=type(str()) and type(raw_data_20[h_list[j]][i])==type(str())) : 
      temp.append(raw_data_20[name][i])
    elif (type(raw_data_20[name][i])==type(str()) and type(raw_data_20[h_list[j]][i])==type(str())) : 
      temp.append("No")
    else : 
      avg = (raw_data_20[name][i]+raw_data_20[h_list[j]][i])/2
      temp.append(avg)
    j += 1
  en_list.append(temp)

In [ ]:
raw_data_20[['전문성_평균평점', '리더십_평균평점', '부하육성_평균평점', '관계관리_평균평점', '조직비전_평균평점', '직책수행능력_평균평점', '조직내 필요성_평균평점', '동반자 의식_평균평점']] = en_list
raw_data_20 = raw_data_20.drop(f_list, axis=1)
raw_data_20 = raw_data_20.drop(h_list, axis=1)

In [ ]:
tmp = pd.DataFrame(columns=raw_data_20.columns)
for name in sen_2020 : 
  temp = raw_data_20.loc[raw_data_20['사번']==name]
  tmp = pd.concat([tmp, temp], ignore_index=True, axis=0)

In [ ]:
tmp = tmp.join(sentiment_2020.set_index('사번')['직책수행능력'], on='사번')
tmp = tmp.join(sentiment_2020.set_index('사번')['조직내 필요성'], on='사번')
tmp = tmp.join(sentiment_2020.set_index('사번')['동반자 의식'], on='사번')

In [ ]:
data_2020 = tmp
print(len(data_2020))

45


In [ ]:
temp_20 = raw_data_20
b = data_2020['사번'].tolist()

for i in range(len(raw_data_20)) : 
  if(raw_data_20['사번'][i] in b) : 
    temp_20 = temp_20.drop([i], axis=0)

temp_20 = temp_20.loc[temp_20['당년 역량평가등급']!='미입사']
temp_20 = temp_20.loc[temp_20['당년 역량평가등급']!='미평가']

In [ ]:
table = []

for i in range(len(raw_data_20)) : 
  sen = []
  if (raw_data_20['사번'][i] in data_2020['사번'].unique()) : 
    temp = data_2020.loc[data_2020['사번']==raw_data_20['사번'][i]].reset_index(drop=True)
    sen.append(temp['직책수행능력'][0])
    sen.append(temp['조직내 필요성'][0])
    sen.append(temp['동반자 의식'][0])
  else : 
    sen.append('No')
    sen.append('No')
    sen.append('No')
  table.append(sen)

In [ ]:
raw_data_20[['직책수행능력', '조직내 필요성', '동반자 의식']] = table

In [ ]:
raw_data_21['전공'] = major['대분류']

In [ ]:
tmp = pd.DataFrame(columns=raw_data_21.columns)
for name in sen_2021 : 
  temp = raw_data_21.loc[raw_data_21['사번']==name]
  tmp = pd.concat([tmp, temp], ignore_index=True, axis=0)

In [ ]:
tmp = tmp.join(sentiment_2021.set_index('사번')['직책수행능력'], on='사번')
tmp = tmp.join(sentiment_2021.set_index('사번')['조직내 필요성'], on='사번')
tmp = tmp.join(sentiment_2021.set_index('사번')['동반자 의식'], on='사번')

In [ ]:
data_2021 = tmp
print(len(data_2021))

53


In [ ]:
temp_21 = raw_data_21
b = data_2021['사번'].tolist()

for i in range(len(raw_data_21)) : 
  if(raw_data_21['사번'][i] in b) : 
    temp_21 = temp_21.drop([i], axis=0)

temp_21 = temp_21.loc[temp_21['전문성_평균평점']!='미입사']
temp_21 = temp_21.loc[temp_21['전문성_평균평점']!='미평가']

In [ ]:
table = []

for i in range(len(raw_data_21)) : 
  sen = []
  if (raw_data_21['사번'][i] in data_2021['사번'].unique()) : 
    temp = data_2021.loc[data_2021['사번']==raw_data_21['사번'][i]].reset_index(drop=True)
    sen.append(temp['직책수행능력'][0])
    sen.append(temp['조직내 필요성'][0])
    sen.append(temp['동반자 의식'][0])
  else : 
    sen.append('No')
    sen.append('No')
    sen.append('No')
  table.append(sen)

In [ ]:
raw_data_21[['직책수행능력', '조직내 필요성', '동반자 의식']] = table

In [ ]:
metric_feature = ['전문성_평균평점', '리더십_평균평점', '부하육성_평균평점', '관계관리_평균평점',
       '조직비전_평균평점', '직책수행능력_평균평점', '조직내 필요성_평균평점', '동반자 의식_평균평점']

temp_19 = temp_19.drop(metric_feature, axis=1)
temp_20 = temp_20.drop(metric_feature, axis=1)
temp_21 = temp_21.drop(metric_feature, axis=1)

In [ ]:
data_2019 = data_2019.sort_values(by='사번')
data_2020 = data_2020.sort_values(by='사번')
data_2021 = data_2021.sort_values(by='사번')

In [ ]:
temp_19 = temp_19.sort_values(by='사번')
temp_20 = temp_20.sort_values(by='사번')
temp_21 = temp_21.sort_values(by='사번')

In [ ]:
user_data_19 = data_2019['사번'].tolist()
user_temp_19 = temp_19['사번'].tolist()
user_all_19 = raw_data_19['사번'].tolist()


user_all_19 = [x for x in user_all_19 if x not in user_data_19]
user_all_19 = [x for x in user_all_19 if x not in user_temp_19]

print(len(user_all_19) + len(user_data_19) + len(user_temp_19))

index_19 = []
for i in range(len(raw_data_19)) :
  for name in user_all_19 : 
    if raw_data_19['사번'][i]==name : 
      index_19.append(i)

no_19 = raw_data_19.iloc[index_19]
no_19 = no_19.drop(metric_feature, axis=1)
no_19 = no_19.drop(['직책수행능력', '조직내 필요성', '동반자 의식'], axis=1)
temp_19 = pd.concat([temp_19, no_19], axis=0)

296


In [ ]:
user_data_20 = data_2020['사번'].tolist()
user_temp_20 = temp_20['사번'].tolist()
user_all_20 = raw_data_20['사번'].tolist()


user_all_20 = [x for x in user_all_20 if x not in user_data_20]
user_all_20 = [x for x in user_all_20 if x not in user_temp_20]

print(len(user_all_20) + len(user_data_20) + len(user_temp_20))

index_20 = []
for i in range(len(raw_data_20)) :
  for name in user_all_20 : 
    if raw_data_20['사번'][i]==name : 
      index_20.append(i)

no_20 = raw_data_20.iloc[index_20]
no_20 = no_20.drop(metric_feature, axis=1)
no_20 = no_20.drop(['직책수행능력', '조직내 필요성', '동반자 의식'], axis=1)
temp_20 = pd.concat([temp_20, no_20], axis=0)

296


In [ ]:
user_data_21 = data_2021['사번'].tolist()
user_temp_21 = temp_21['사번'].tolist()
user_all_21 = raw_data_21['사번'].tolist()


user_all_21 = [x for x in user_all_21 if x not in user_data_21]
user_all_21 = [x for x in user_all_21 if x not in user_temp_21]

print(len(user_all_21) + len(user_data_21) + len(user_temp_21))

index_21 = []
for i in range(len(raw_data_21)) :
  for name in user_all_21 : 
    if raw_data_21['사번'][i]==name : 
      index_21.append(i)

no_21 = raw_data_21.iloc[index_21]
no_21 = no_21.drop(metric_feature, axis=1)
no_21 = no_21.drop(['직책수행능력', '조직내 필요성', '동반자 의식'], axis=1)
temp_21 = pd.concat([temp_21, no_21], axis=0)

296


In [ ]:
len(user_all_21)

43

In [ ]:
raw_data_22 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/추가데이터/NH농협캐피탈 데이터 작업_승진적합_1114.xlsx', sheet_name=3)

In [ ]:
raw_data_22

,사번,성명,연락처,이메일,생년월일,본부,실,팀,직무,직급,...,수상횟수,평균 교육점수,교육 이수여부,당년 역량평가등급,당년 업적평가등급,역량평가등급(1년전),역량평가등급(2년전),업적평가등급(1년전),업적평가등급(2년전),승진대상 후보여부
0,B0810003,변홍구,010-9570-2340,byunhg@nhcapital.co.kr,1964-05-12,리스크관리본부,리스크관리실,리스크검증팀,리스크(여신감리역),업무지원역,...,1,교육 진행중,교육 진행중,B,A,미평가,미평가,미평가,미평가,No
1,B0810004,나혁준,010-8637-8325,hjna@nhcapital.co.kr,1975-11-26,전략금융본부,전략사업실,FLEET금융팀,부서총괄,부장,...,1,교육 진행중,교육 진행중,B,A,B,A,B,A,Yes
2,B0810007,이창준,010-6532-3351,cjlee@nhcapital.co.kr,1972-02-20,디지털금융본부,DT추진센터,고객지원팀,영업기획·지원(CS/청구수납),차장,...,1,교육 진행중,교육 진행중,C,B,C,B,B,B,Yes
3,B0810011,김규오,010-3767-5867,gokim@nhcapital.co.kr,1969-06-06,NaN,금융소비자보호총괄책임자,NaN,부서총괄,이사대우,...,1,교육 진행중,교육 진행중,A,A,A,B,A,C,No
4,B0810012,이유,010-2881-7747,leeu@nhcapital.co.kr,1969-06-28,디지털금융본부,NaN,NaN,부서총괄,이사대우,...,2,교육 진행중,교육 진행중,A,A,S,S,S,S,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,B2210098,양동섭,010-5184-0927,ds0927@nhcapital.co.kr,1983-09-27,NaN,준법감시인,준법지원팀,내부통제(준법감시-변호사),과장,...,미입사,교육 진행중,교육 진행중,미대상,미대상,미입사,미입사,미입사,미입사,No
292,B2210101,정연호,010-4416-2891,jyh86@nhcapital.co.kr,1986-09-16,여신관리본부,여신심사실,기업심사팀,여신심사(기업/투자/일반리스/Fleet금융),과장,...,미입사,교육 진행중,교육 진행중,미대상,미대상,미입사,미입사,미입사,미입사,No
293,B2210109,송민경,010-5474-3925,smk0119@nhcapital.co.kr,1982-01-19,리테일금융본부,리테일금융실,리테일기획팀,영업기획·지원(리테일금융),과장,...,미입사,교육 진행중,교육 진행중,미대상,미대상,미입사,미입사,미입사,미입사,No
294,B2210110,채유리,010-7524-3120,youreec1@nhcapital.co.kr,1986-04-08,여신관리본부,여신심사실,리테일심사팀,여신심사(오토금융),대리,...,미입사,교육 진행중,교육 진행중,미대상,미대상,미입사,미입사,미입사,미입사,No


In [ ]:
data_2022 = data_2021.copy()
temp_22 = temp_21.copy()

for name in raw_data_22.columns.tolist() : 
  if name != '사번' : 
    data_2022 = data_2022.join(raw_data_22.set_index('사번')[name], on='사번', rsuffix='_2022')
    temp_22 = temp_22.join(raw_data_22.set_index('사번')[name], on='사번', rsuffix='_2022')

# 21년 데이터는 제거하고 22년 데이터를 사용하되 교육점수, 교육 이수 여부에 대해서는 21년 데이터를 살리는 방향으로 구성

In [ ]:
data_2022 = data_2022[['사번','성명_2022', '연락처_2022', '이메일_2022', '생년월일_2022', '본부_2022', '실_2022', '팀_2022',
       '직무_2022', '직급_2022', '직책_2022', '성별_2022', '연령_2022',
       '당사근속기간(개월)_2022', '외부경력(개월)_2022', '총경력(개월)_2022', '현 직급 근속기간(연)_2022',
       '이직횟수_2022', '입사경로_2022', '학력_2022', '전공', '거주지_2022',
       '셀 참여횟수_2022', '봉사활동 참여횟수_2022', '직무관련자격증_2022', '수상횟수_2022',
       '평균 교육점수', '교육 이수여부', '당년 역량평가등급_2022', '당년 업적평가등급_2022',
       '역량평가등급(1년전)_2022', '역량평가등급(2년전)_2022', '업적평가등급(1년전)_2022',
       '업적평가등급(2년전)_2022', '승진대상 후보여부_2022', '전문성_평균평점', '리더십_평균평점',
       '부하육성_평균평점', '관계관리_평균평점', '조직비전_평균평점', '직책수행능력_평균평점', '조직내 필요성_평균평점',
       '동반자 의식_평균평점', '직책수행능력', '조직내 필요성', '동반자 의식']]

data_2022.columns = ['사번', '성명', '연락처', '이메일', '생년월일', '본부', '실', '팀', '직무', '직급', '직책', '성별',
       '연령', '당사근속기간(개월)', '외부경력(개월)', '총경력(개월)', '현 직급 근속기간(연)', '이직횟수',
       '입사경로', '학력', '전공', '거주지', '셀 참여횟수', '봉사활동 참여횟수', '직무관련자격증', '수상횟수',
       '평균 교육점수', '교육 이수여부', '당년 역량평가등급', '당년 업적평가등급', '역량평가등급(1년전)',
       '역량평가등급(2년전)', '업적평가등급(1년전)', '업적평가등급(2년전)', '승진대상 후보여부', '전문성_평균평점', '리더십_평균평점',
       '부하육성_평균평점', '관계관리_평균평점', '조직비전_평균평점', '직책수행능력_평균평점', '조직내 필요성_평균평점',
       '동반자 의식_평균평점', '직책수행능력', '조직내 필요성', '동반자 의식']

In [ ]:
temp_22 = temp_22[['사번','성명_2022', '연락처_2022', '이메일_2022', '생년월일_2022', '본부', '실', '팀',
       '직무_2022', '직급_2022', '직책_2022', '성별_2022', '연령_2022',
       '당사근속기간(개월)_2022', '외부경력(개월)_2022', '총경력(개월)_2022', '현 직급 근속기간(연)_2022',
       '이직횟수_2022', '입사경로_2022', '학력_2022', '전공', '거주지_2022',
       '셀 참여횟수_2022', '봉사활동 참여횟수_2022', '직무관련자격증_2022', '수상횟수_2022',
       '평균 교육점수', '교육 이수여부', '당년 역량평가등급_2022', '당년 업적평가등급_2022',
       '역량평가등급(1년전)_2022', '역량평가등급(2년전)_2022', '업적평가등급(1년전)_2022',
       '업적평가등급(2년전)_2022', '승진대상 후보여부_2022']]

temp_22.columns = ['사번', '성명', '연락처', '이메일', '생년월일', '본부', '실', '팀', '직무', '직급', '직책', '성별',
       '연령', '당사근속기간(개월)', '외부경력(개월)', '총경력(개월)', '현 직급 근속기간(연)', '이직횟수',
       '입사경로', '학력', '전공', '거주지', '셀 참여횟수', '봉사활동 참여횟수', '직무관련자격증', '수상횟수',
       '평균 교육점수', '교육 이수여부', '당년 역량평가등급', '당년 업적평가등급', '역량평가등급(1년전)',
       '역량평가등급(2년전)', '업적평가등급(1년전)', '업적평가등급(2년전)', '승진대상 후보여부']

In [ ]:
data_2019.to_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합/promotion_subject_19.xlsx', index=False)
data_2020.to_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합/promotion_subject_20.xlsx', index=False)
data_2021.to_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합/promotion_subject_21.xlsx', index=False)
data_2022.to_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합/promotion_subject_22.xlsx', index=False)

In [ ]:
mind = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/dataset/인적성 검사 데이터_1117.xlsx')

In [ ]:
mind_data = mind.copy()
temp_19_m = temp_19.copy()
temp_20_m = temp_20.copy()
temp_21_m = temp_21.copy()
temp_22_m = temp_22.copy()

In [ ]:
mind_feature = ['긍정성', '적극성', '성실성', '사회성', '정체성', '관계성', '객관성',
       '비활성', '과활성', '자기긍정', '기회긍정', '환경긍정', '성취추구', '인정추구', '성장추구', '점검행동',
       '조절행동', '유지행동', '감정인식', '맥락이해', '관계대응', '성찰성', '수용성', '능동성', '공감성',
       '우호성', '협력성', '도덕성', '정직성', '책임성', '우울성(저항성)', '부정성(저항성)', '태만성(저항성)', '불안성(저항성)', '충동성(저항성)',
       '공격성(저항성)', '역량 평균']

for name in mind_feature : 
  temp_19_m = temp_19_m.join(mind_data.set_index('이메일')[name], on='이메일')
  temp_20_m = temp_20_m.join(mind_data.set_index('이메일')[name], on='이메일')
  temp_21_m = temp_21_m.join(mind_data.set_index('이메일')[name], on='이메일')
  temp_22_m = temp_22_m.join(mind_data.set_index('이메일')[name], on='이메일')

In [ ]:
temp_19_m.to_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합/promotion_mind_19.xlsx', index=False)
temp_20_m.to_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합/promotion_mind_20.xlsx', index=False)
temp_21_m.to_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합/promotion_mind_21.xlsx', index=False)
temp_22_m.to_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합/promotion_mind_22.xlsx', index=False)